<img src="img/csdms_logo.jpg">

# BMI Visualize!

Visualize the output of the BMI-ed Diffusions model as a function of time.

## Setup

Check that these packages are installed on your system:

* pandas
* matplotlib
* bokeh
* xarray
* holoviews

Some imports to start, including `holoviews`:

In [ ]:
import os
import numpy as np
import holoviews as hv
hv.extension('bokeh', 'matplotlib')

## Load and run the Diffusion model through its BMI

Start by importing the `BmiDiffusion` class from the **bmi-live-2018** package and creating an instance:

In [ ]:
from bmi_live.bmi_diffusion import BmiDiffusion
x = BmiDiffusion()

Locate a configuration file for this example and use it to initialize the model:

In [ ]:
from bmi_live import data_directory
cfg_file = os.path.join(data_directory, 'visualize_diffusion.yaml')
x.initialize(cfg_file)

Get a reference to the temperature variable exposed by the model:

In [ ]:
temp = x.get_value_ref('plate_surface__temperature')

Add a unit impulse to the initial temperature field: 

In [ ]:
temp[200] = 100.0

Because `temp` is a reference, there's no need to call `set_value` to update its value.

Get the id of the grid on which the temperature variable is located:

In [ ]:
grid_id = x.get_var_grid('plate_surface__temperature')
grid_id

Use this grid id to get the shape of the grid:

In [ ]:
grid_shape = x.get_grid_shape(grid_id)
grid_shape

Set the number of time steps that we'll advance the model:

In [ ]:
n_steps = 35

Make a variable to hold the values of the temperature field over time:

In [ ]:
temperature = np.zeros((*grid_shape, n_steps))

What is the shape of this variable?

In [ ]:
temperature.shape

Load the initial perturbed temperature field into the `temperature` variable:

In [ ]:
temperature[:,:,0] = temp.reshape(grid_shape)

Advance the model through time, adding to the `temperature` array at each step:

In [ ]:
for t in range(n_steps):
    x.update()
    temperature[:,:,t] = temp.reshape(grid_shape)

Note that because `temp` is a reference, it's automatically updated on each time step.

## Visualize output from the Diffusion model

Load the `temperature` array into a `holoviews` dataset.

In [ ]:
ds = hv.Dataset((np.arange(n_steps), np.arange(grid_shape[1]), np.arange(grid_shape[0]),
                 temperature), ['Time','x', 'y'], 'Temperature')

Examine the dataset:

In [ ]:
type(ds.data), list(ds.data.keys())

Visualize the dataset as an image with a slider to step through time:

In [ ]:
%opts Image (cmap='Reds')[colorbar=True]
%output size=150
ds.to(hv.Image, ['x', 'y'], dynamic=True)